In [ ]:
# Test LLM API call and caching
import asyncio
from evals.llm_api import call_llm_api

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the capital of France?"}
]

model = "anthropic/claude-sonnet-4-20250514"

async def run_test():
    result = await call_llm_api(
        messages=messages,
        model=model,
        temperature=0.2,
        max_tokens=128,
        caching=False,
    )
    print("\n\n\nResponse:", result.response_text)
    print("\n\n\nError:", result.error)

# Run twice to test caching
await run_test()

INFO:evals.llm_api:Attempting to call model: anthropic/claude-sonnet-4-20250514
15:02:16 - LiteLLM:INFO: utils.py:2827 - 
LiteLLM completion() model= claude-sonnet-4-20250514; provider = anthropic
INFO:LiteLLM:
LiteLLM completion() model= claude-sonnet-4-20250514; provider = anthropic


INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 400 Bad Request"
INFO:evals.llm_api:Attempting to call fallback model: openrouter/anthropic/claude-sonnet-4
15:02:16 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: anthropic/claude-sonnet-4
INFO:evals.llm_api:Successfully received response from openrouter/anthropic/claude-sonnet-4
INFO:LiteLLM:selected model name for cost calculation: anthropic/claude-sonnet-4
15:02:16 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: anthropic/claude-sonnet-4
INFO:LiteLLM:selected model name for cost calculation: anthropic/claude-sonnet-4
15:02:16 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: anthropic/claude-sonnet-4
INFO:LiteLLM:selected model name for cost calculation: anthropic/claude-sonnet-4
15:02:16 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: anthropic/claude-sonnet-4
INFO:Li


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.




Response: The capital of France is Paris.



Error: None


In [2]:
await run_test()

INFO:evals.llm_api:Attempting to call model: anthropic/claude-sonnet-4-20250514
14:50:11 - LiteLLM:INFO: utils.py:2827 - 
LiteLLM completion() model= claude-sonnet-4-20250514; provider = anthropic
INFO:LiteLLM:
LiteLLM completion() model= claude-sonnet-4-20250514; provider = anthropic
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 400 Bad Request"
INFO:evals.llm_api:Attempting to call fallback model: openrouter/anthropic/claude-sonnet-4
14:50:11 - LiteLLM:INFO: utils.py:2827 - 
LiteLLM completion() model= anthropic/claude-sonnet-4; provider = openrouter
INFO:LiteLLM:
LiteLLM completion() model= anthropic/claude-sonnet-4; provider = openrouter



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
14:50:13 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: anthropic/claude-sonnet-4
INFO:LiteLLM:selected model name for cost calculation: anthropic/claude-sonnet-4
INFO:evals.llm_api:Successfully received response from openrouter/anthropic/claude-sonnet-4





Response: The capital of France is Paris.



Error: None


14:50:13 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: anthropic/claude-sonnet-4
INFO:LiteLLM:selected model name for cost calculation: anthropic/claude-sonnet-4
